In [1]:
import requests
import pandas as pd

In [2]:
base_url = "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/georef-france-commune/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"
response = requests.get(base_url)
response.raise_for_status()

In [3]:
filename = "georef-france-commune_all.csv"
with open(filename, "wb") as f:
    f.write(response.content)
print(f"Downloaded {filename}")

Downloaded georef-france-commune_all.csv


In [10]:
data = pd.read_csv(
    filename,
    delimiter=";",
    usecols=[
        "Geo Point",
        "Nom Officiel Département",
        "Nom Officiel Commune",
        "Code Officiel Commune",
    ],
)

data.columns

Index(['Geo Point', 'Nom Officiel Département', 'Code Officiel Commune',
       'Nom Officiel Commune'],
      dtype='object')

In [9]:
data.filter(like="Code").head()

,Code Officiel Région,Code Officiel Département,Code Officiel Arrondissement départemental,Code Officiel Zone emploi 2020,Code Officiel Bassin vie 2022,Code Officiel EPCI,Code Officiel EPT,Code Officiel Commune,Code Officiel Courant Commune,Code Iso 3166-3 Zone,Code Officiel Zone emploi 2010,Code CATAEU2010,Code UU2010,Code AU2010,Code CATEAAV2020,Code UU2020,Code AAV2020,Code Canton Ville
0,93,04,043,9307.0,04070,200067437.0,NaN,04122,04122,FXX,NaN,NaN,NaN,NaN,20.0,04000,251,0405
1,93,04,044,9311.0,04088,240400440.0,NaN,04164,04164,FXX,NaN,NaN,NaN,NaN,30.0,04000,000,0406
2,93,04,044,9311.0,84003,200071025.0,NaN,04175,04175,FXX,NaN,NaN,NaN,NaN,20.0,04000,275,0411
3,93,05,052,9310.0,05061,200067320.0,NaN,05115,05115,FXX,NaN,NaN,NaN,NaN,20.0,05000,119,0504
4,93,05,051,9302.0,38052,240500439.0,NaN,05181,05181,FXX,NaN,NaN,NaN,NaN,30.0,05000,000,0502


In [12]:
populations = pd.read_csv(
    "./base-pop-historiques2021.csv",
    sep=";",
)
populations

,CODGEO,REG,DEP,LIBGEO,PMUN2021
0,01001,84,01,L'Abergement-Clémenciat,832
1,01002,84,01,L'Abergement-de-Varey,267
2,01004,84,01,Ambérieu-en-Bugey,14 854
3,01005,84,01,Ambérieux-en-Dombes,1 897
4,01006,84,01,Ambléon,113
...,...,...,...,...,...
34942,97420,4,974,Sainte-Suzanne,24 293
34943,97421,4,974,Salazie,7 243
34944,97422,4,974,Le Tampon,81 943
34945,97423,4,974,Les Trois-Bassins,6 899


In [26]:
data_with_population = data.join(
    populations.set_index("CODGEO")[["PMUN2021"]], on="Code Officiel Commune"
)

In [27]:
# split Geo Point into latitude and longitude
data_with_population[["latitude", "longitude"]] = data_with_population[
    "Geo Point"
].str.split(",", expand=True)
data_with_population = data_with_population.drop(columns=["Geo Point"])
data_with_population["latitude"] = (
    data_with_population["latitude"].astype(float).round(2)
)
data_with_population["longitude"] = (
    data_with_population["longitude"].astype(float).round(2)
)
data_with_population.rename(columns={"PMUN2021": "population"}, inplace=True)
data_with_population

,Nom Officiel Département,Code Officiel Commune,Nom Officiel Commune,population,latitude,longitude
0,Alpes-de-Haute-Provence,04122,Mirabeau,509,44.06,6.08
1,Alpes-de-Haute-Provence,04164,Revest-Saint-Martin,85,44.02,5.82
2,Alpes-de-Haute-Provence,04175,Sainte-Croix-à-Lauze,88,43.90,5.62
3,Hautes-Alpes,05115,Remollon,463,44.47,6.16
4,Hautes-Alpes,05181,Villar-d'Arêne,283,45.00,6.36
...,...,...,...,...,...,...
34953,Hautes-Pyrénées,65195,Génos,136,42.75,0.39
34954,Hautes-Pyrénées,65354,Pailhac,79,42.91,0.37
34955,Hautes-Pyrénées,65453,Troubat,75,42.97,0.58
34956,Hautes-Pyrénées,65467,Vier-Bordes,95,42.99,-0.03


In [28]:
data_with_population.to_parquet("georef-france-commune.parquet")

In [29]:
data_with_population.head()

,Geo Point,Nom Officiel Département,Code Officiel Commune,Nom Officiel Commune
0,"44.05663317511026, 6.080498812220846",Alpes-de-Haute-Provence,04122,Mirabeau
1,"44.019569849537234, 5.823167584619715",Alpes-de-Haute-Provence,04164,Revest-Saint-Martin
2,"43.90416218986043, 5.616017596751714",Alpes-de-Haute-Provence,04175,Sainte-Croix-à-Lauze
3,"44.471613451136896, 6.1595420491675075",Hautes-Alpes,05115,Remollon
4,"45.00016914643277, 6.36413066153131",Hautes-Alpes,05181,Villar-d'Arêne


In [30]:
data_with_population.query("`Nom Officiel Commune` == 'Craponne'")

,Geo Point,Nom Officiel Département,Code Officiel Commune,Nom Officiel Commune
23515,"45.746009253446125, 4.726884209654968",Rhône,69069,Craponne


In [32]:
data_with_population.to_csv("georef-france-commune.csv", index=False)

# Reindex from grid points

In [ ]:
from pollen_forecast.copernicus import PollenForcastCopernicusGeneric

In [ ]:
my_pollen = PollenForcastCopernicusGeneric(
    north=51.70, south=41.87, east=8.74, west=-5.33, prefix="./france_territory/"
)
my_pollen.get_pollen_data()

In [ ]:
import xarray as xr

data = xr.open_dataset(my_pollen.filename)
data.coords["longitude"] = (data.coords["longitude"] + 180) % 360 - 180
data_sorted = data.sortby("longitude")
data_sorted

In [ ]:
import cartopy
from cartopy import crs as ccrs
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={"projection": ccrs.PlateCarree()})
data_sorted["gpg_conc"].isel(time=0, level=0).plot(
    ax=ax,
    transform=ccrs.PlateCarree(),
)

ax.coastlines()
# add borders
ax.add_feature(cartopy.feature.BORDERS, linestyle=":")

In [ ]:
communes = pd.read_csv("georef-france-commune.csv")
communes.head()

In [ ]:
# find the neasrest grip point of the Xarray Dataset for each commune
def find_nearest_grip_point(lat, long, data):
    print(f"Finding nearest grip point for {lat}, {long}")
    point = data.sel(latitude=lat, longitude=long, method="nearest")
    try:
        lat_nearest = point.latitude.values
        long_nearest = point.longitude.values
    except IndexError:
        print(f"Error with {lat}, {long}")
        print(point.latitude.values)
        print(point.longitude.values)
        return None, None

    return lat_nearest, long_nearest


communes["latitude"], communes["longitude"] = zip(
    *communes.apply(
        lambda x: find_nearest_grip_point(
            x["Geo Point"].split(",")[0], x["Geo Point"].split(",")[1], data_sorted
        ),
        axis=1,
    )
)
communes.head()

In [ ]:
communes.to_csv("georef-france-commune.csv", index=False)

In [ ]:
ax = (
    data_sorted["gpg_conc"]
    .sel(level=0, longitude=2.35, latitude=48.85, method="nearest")
    .plot()
)

In [ ]:
data_sorted["gpg_conc"].sel(
    level=0, latitude=4.85, longitude=45.75, method="nearest"
).plot()

In [1]:
# add Prefecture Labelle

In [2]:
from pathlib import Path
import pandas as pd

In [39]:

root_dir = Path("../data")
georef_filename = root_dir / "georef-france-commune.csv"
prefectures_filename =  root_dir / "prefectures.csv"

georef_df = pd.read_csv(georef_filename)[["Nom Officiel Département", "Nom Officiel Commune", "latitude", "longitude"]]
georef_df.head()

,Nom Officiel Département,Nom Officiel Commune,latitude,longitude
0,Alpes-de-Haute-Provence,Mirabeau,44.05,6.050003
1,Alpes-de-Haute-Provence,Revest-Saint-Martin,44.05,5.850006
2,Alpes-de-Haute-Provence,Sainte-Croix-à-Lauze,43.95,5.649994
3,Hautes-Alpes,Remollon,44.45,6.149994
4,Hautes-Alpes,Villar-d'Arêne,45.05,6.350006


In [25]:
prefectures_df = pd.read_csv(prefectures_filename, sep=";")
prefectures_df.head()

,Geo Point,Geo Shape,gid,Code INSEE,Commune,Service
0,"44.38771428187486, 6.643645611399548","{""coordinates"": [6.643645611399548, 44.3877142...",13,04019,Barcelonnette,Sous-préfecture
1,"49.845842844647706, 3.290619591335112","{""coordinates"": [3.290619591335112, 49.8458428...",7,02691,Saint-Quentin,Sous-préfecture
2,"44.894321981740646, 6.632905771977145","{""coordinates"": [6.632905771977145, 44.8943219...",17,05023,Briançon,Sous-préfecture
3,"45.75813629221574, 5.686648106090643","{""coordinates"": [5.686648106090643, 45.7581362...",1,01034,Belley,Sous-préfecture
4,"48.585217460500026, 7.736631137529066","{""coordinates"": [7.736631137529066, 48.5852174...",239,67482,Strasbourg,Préfecture de région


In [40]:
georef_df["is_prefecture"] = False
for name in prefectures_df["Commune"]:
    mask = georef_df["Nom Officiel Commune"].str.lower() == name.lower()
    if sum(mask) == 1:
        georef_df.loc[mask, "is_prefecture"] = True
    elif sum(mask) > 1:
        lat, lon = prefectures_df.loc[prefectures_df["Commune"] == name, "Geo Point"].item().split(",")
        lat, lon = float(lat), float(lon)
        min_distance = 999999999
        for idx, row in georef_df[mask].iterrows():
            distance = ( (row["latitude"] - lat) ** 2 + (row["longitude"] - lon) ** 2 ) ** 0.5
            if distance < min_distance:
                min_idx = idx
                min_name = row["Nom Officiel Commune"]
                min_distance = distance
        georef_df.loc[min_idx, "is_prefecture"] = True
    else:
        print("no match", name)
        

no match Château-Gontier
no match Etampes
no match Saint-Etienne
no match Ancenis
no match L'Hay-les-roses
no match Le-Puy-en-Velay
no match Sainte-Ménehould
no match Evry
no match Florac
no match St-Jean-d'Angély
no match Luneville
no match Sègre
no match Châlon-sur-saône
no match Vire
no match Cherbourg-Octeville
no match Epernay
no match Daint-Dié-des-Vosges
no match Epinal
no match Evreux
no match Briey


In [41]:
georef_df.to_csv(georef_filename)

In [38]:
georef_df

,Geo Point,Nom Officiel Département,Nom Officiel Commune,latitude,longitude,is_prefecture,"(11930, is_prefecture)","(24535, is_prefecture)","(10151, is_prefecture)","(31805, is_prefecture)",...,"(9230, is_prefecture)","(29755, is_prefecture)","(1081, is_prefecture)","(33065, is_prefecture)","(34291, is_prefecture)","(2495, is_prefecture)","(33838, is_prefecture)","(32583, is_prefecture)","(14320, is_prefecture)","(3193, is_prefecture)"
0,"44.05663317511026, 6.080498812220846",Alpes-de-Haute-Provence,Mirabeau,44.05,6.050003,False,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1,"44.019569849537234, 5.823167584619715",Alpes-de-Haute-Provence,Revest-Saint-Martin,44.05,5.850006,False,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2,"43.90416218986043, 5.616017596751714",Alpes-de-Haute-Provence,Sainte-Croix-à-Lauze,43.95,5.649994,False,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
3,"44.471613451136896, 6.1595420491675075",Hautes-Alpes,Remollon,44.45,6.149994,False,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
4,"45.00016914643277, 6.36413066153131",Hautes-Alpes,Villar-d'Arêne,45.05,6.350006,False,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34953,"42.751905635116614, 0.39128298917933746",Hautes-Pyrénées,Génos,42.75,0.350006,False,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
34954,"42.91190408731917, 0.3672848379459593",Hautes-Pyrénées,Pailhac,42.95,0.350006,False,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
34955,"42.974991073911966, 0.5817118922968775",Hautes-Pyrénées,Troubat,42.95,0.550003,False,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
34956,"42.9934225272069, -0.031115626290251937",Hautes-Pyrénées,Vier-Bordes,42.95,-0.049988,False,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
